<a href="https://colab.research.google.com/github/shramee/Fiat-Shamir-and-Schwartz-Zippel-for-Faster-Emulated-pairings/blob/main/Fiat_Shamir_and_Schwartz_Zippel_for_Faster_Emulated_pairings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fiat-Shamir and Schwartz-Zippel for Faster Emulated Pairings

## 1. Introduction

In the write-up [Faster Extension Field multiplications for Emulated Pairing Circuits](https://hackmd.io/@feltroidprime/B1eyHHXNT#fn3), Feltroid Prime discusses using Schwartz Zippel lemma and Fiat Shamir heuristics to optimize Field multiplications (Sections 1, 2 and 3).

Starting with representing $𝔽_{p12}$ (and equivalently $𝔽_{p6}$) as direct extensions in section 1. Then representing multiplication operation as a polynomial in section 2 (equation 1). Finally using Fiat-Shamir and the Schwartz-Zippel lemma in the circuit to verify the computation.

> ${A(x)*B(x) = Q(x)*P(x) + R(x)}$ &emsp; &emsp; &emsp;${(1)}$

We will expand on this to apply it to larger computations. To start with we will try to adapt this for exponentiation (equation 2), with a goal of replacing entire final exponentiation.

> ${A(x)^n = Q(x)*P(x) + R(x)}$ &emsp; &emsp; &emsp;${(2)}$

## 2. Challenges

### 2.1 Controling polynomial degree

As we try to prove the larger computations, exponentiation of an $𝔽_{p12}$ by n, the degree of ${Q(x)}$ grows linearly with n.
With a large degree comes increased calldata size and computation costs for hashing each coefficient for Fiat-Shamir. To make things easier we prefer an optimised version for larger computations (equation 3).

> $A(x)^n = R(x)$ $mod$ $P(x)$ &emsp; &emsp; &emsp; &emsp;$(3)$

### 2.2 Computation of Fiat-Shamir, $z$

Details regarding the computation of Fiat Shamir hueristic are presented here, [Section 3 of FEFMEPC](https://hackmd.io/@feltroidprime/B1eyHHXNT#3-Using-Fiat-Shamir-and-the-Schwartz-Zippel-lemma-in-circuit).
To have our evaluations mod ${P(x)}$ we need to reduce our Fiat-Shamir hash, $z$, so that computation of ${P(z)} doesn't require reduction.

With ${P(z)}$ for modulo, bitsize of `z` from Fiat Shamir needs to be reduced to $1/(deg P(x))$ of maximum bitsize available for a modulo operation.


### 2.3. Size of coefficients

The size of the coefficients in bits grows linearly with $n$, the exponent.
This can be managed easily by modding, but here's the challenge.
1. We need the coefficients modulo curve's prime field $p$. Otherwise the result is not usable for us in pairing.
2. The evaluation requires a very specific modulus based on result of Fiat-Shamir otherwise the equation doesn't hold. Let's say we need it mod $z_{field} = P(x)$.

So we have the prover provide the coefficients mod $z_{field} . p$

For our purposes where `n` might be well over a hundred of bits this poses a challenge.

